# Exercise #2

This notebook is created by **Chinchuthakun Worameth** as a part of **Sparse Signal Processing and Optimization (ART.T465)** at Tokyo Institute of Technology taught in Fall semester 2021 by Prof. Ono Shunsuke. It contains
- Implementation of proximal operator for the L1 norm and the nuclear norm
- Implementation of projection onto the box contraint

## 1. Proximal Operator

**Proximal operator** of function $f$ is defined as
$$
\text{prox}_{\gamma f}(x) = \text{argmin}_y f(y) + \frac{1}{2\gamma}\Vert x - y \Vert^2, \gamma>0
$$

### 1.1. L1 norm
We define **L1 norm** and its proximal operator as
$$
f(x) = \Vert x \Vert_1 \rightarrow \text{prox}_{\gamma f}(x) = \text{SoftThre}(x,\gamma)
$$
such that
$$
\text{SoftThre}(x,\gamma) = \text{sgn}(x) \circ \text{max}(\text{abs}(x), 0)
$$
where $\text{sgn}(x)$, $\circ$, and $\text{abs}(x)$ are the sign function, Hadamard product, and element-wise absolute value of $x$ respectively.

### 1.2. Nuclear norm
Similarly, **Nuclear norm** and its proximal operator are
$$
g(X) = \Vert X \Vert_* \rightarrow \text{prox}_{\gamma g}(X) = U(\text{SoftThre}(\Sigma, \gamma))V^T
$$
where $X=U\Sigma V^T$, i.e. **Singular Value Decomposition (SVD)**.

## 2. Projection



**Convex projection** of $x$ onto a closed convex set $C$ is defined as
$$
P_C(x) = \text{argmin}_{y \in C}\Vert x - y\Vert
$$
Using the indicator function $1_C$ to map points inside and outside $C$ to $0$ and $\infty$, it is obvious that
$$
P_C(x) = \text{prox}_{\gamma 1_C}(x)
$$

### 2.1. Box constraint
Given $C = [a,b]^N$ such that $a < b$, by using the indicator function, we can define the projection onto the box constraint as
$$
P_C(x) = [\max(a,\min(b,x_i))]_i
$$

## 3. Implementation

We define class ```proximalOperator``` for L1 norm, Nuclear norm, and box constraint (since convex projection is equivalent to proximal operator). Then, we demonstrate them on random inputs.

In [ ]:
import numpy as np
import random

In [ ]:
class proximalOperator:
    def __init__(self, param):
        self.param = param
    
    def softThreshold(self, x):
        return np.sign(x) * np.maximum(np.absolute(x) - self.param['gamma'], np.zeros(x.shape))
    
    def calculate(self, x):
        if self.param['name'] == "l1":
            return self.softThreshold(x)
        elif self.param['name'] == "nuclear":
            """
            np.linalg.svd returns s as a vector [s_1, ..., s_r]
            So, we need to convert it to a diagonal matrix first.
            """
            u, s, vt = np.linalg.svd(x)
            s_full = np.zeros(x.shape)
            s_full[:s.shape[0], :s.shape[0]] = np.diag(s)
            return self.softThreshold(s_full)
        elif self.param['name'] == "box":
            all_a = np.full(x.shape, self.param['a'])
            all_b = np.full(x.shape, self.param['b'])
            return np.maximum(all_a, np.minimum(x, all_b))

### 3.1. L1 norm

In [ ]:
param = dict(name = "l1", gamma = 0.5)
proxOp = proximalOperator(param)

In [ ]:
# input is a vector
x = np.random.rand(5)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))
print("#"*20)
# input is a matrix
x = np.random.rand(5, 6)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))

Input:  [0.30641631 0.87921065 0.87514323 0.49088731 0.47492024]
Prox:  [0.         0.37921065 0.37514323 0.         0.        ]
####################
Input:  [[0.40740831 0.2789164  0.85427081 0.18934023 0.07944274 0.38065153]
 [0.56113664 0.81817118 0.36966436 0.84353933 0.74925917 0.75649248]
 [0.19580754 0.34749131 0.71763302 0.05905848 0.45224151 0.10263657]
 [0.60810289 0.21142864 0.98039322 0.6672739  0.91222131 0.93156772]
 [0.86804179 0.3087482  0.03295242 0.97255999 0.65480204 0.4183098 ]]
Prox:  [[0.         0.         0.35427081 0.         0.         0.        ]
 [0.06113664 0.31817118 0.         0.34353933 0.24925917 0.25649248]
 [0.         0.         0.21763302 0.         0.         0.        ]
 [0.10810289 0.         0.48039322 0.1672739  0.41222131 0.43156772]
 [0.36804179 0.         0.         0.47255999 0.15480204 0.        ]]


### 3.2. Nuclear norm

In [ ]:
param = dict(name = "nuclear", gamma = 0.5)
proxOp = proximalOperator(param)

In [ ]:
# input is a matrix
x = np.random.rand(5, 6)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))

Input:  [[0.69102478 0.67771974 0.75383203 0.12139337 0.43796632 0.81449429]
 [0.57403847 0.35781768 0.94001131 0.05257832 0.18693622 0.1612751 ]
 [0.12823358 0.21916698 0.53511912 0.66951926 0.37077236 0.71453993]
 [0.31733218 0.39330334 0.70720833 0.53356012 0.41460748 0.62784016]
 [0.60634103 0.63473826 0.09719439 0.52963226 0.96671707 0.64271493]]
Prox:  [[2.28172365 0.         0.         0.         0.         0.        ]
 [0.         0.453095   0.         0.         0.         0.        ]
 [0.         0.         0.19095044 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]]


### 3.3. Projection onto Box constraint

In [ ]:
param = dict(name = "box", a = .5, b = .8)
proxOp = proximalOperator(param)

In [ ]:
# input is a vector
x = np.random.rand(5)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))
print("#"*20)
# input is a matrix
x = np.random.rand(5, 6)
print("Input: ", x)
print("Prox: ", proxOp.calculate(x))

Input:  [0.98729036 0.06847584 0.47270529 0.47652759 0.05005247]
Prox:  [0.8 0.5 0.5 0.5 0.5]
####################
Input:  [[6.35100864e-01 4.60570568e-01 3.08969271e-01 1.15523384e-01
  3.26515995e-01 1.00685511e-01]
 [6.14512095e-03 8.55313486e-01 6.95162005e-01 5.21968710e-01
  6.06490373e-02 2.33515033e-01]
 [4.37914535e-01 5.03069411e-01 6.58174741e-01 9.91560774e-01
  1.25834953e-01 3.39590721e-01]
 [3.15097338e-01 5.70367942e-01 2.44138841e-01 9.23093198e-01
  5.26741032e-01 2.81680253e-01]
 [9.62101801e-04 1.54559302e-03 3.93889269e-01 6.77582712e-01
  9.96585378e-01 7.12460503e-01]]
Prox:  [[0.63510086 0.5        0.5        0.5        0.5        0.5       ]
 [0.5        0.8        0.695162   0.52196871 0.5        0.5       ]
 [0.5        0.50306941 0.65817474 0.8        0.5        0.5       ]
 [0.5        0.57036794 0.5        0.8        0.52674103 0.5       ]
 [0.5        0.5        0.5        0.67758271 0.8        0.7124605 ]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ceb3b963-7d26-4b9f-840e-3ecd7c45577b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>